In [1]:
url = 'https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page='

In [2]:
import requests 
from bs4 import BeautifulSoup 

In [46]:
class RottenTomatoesScraper(): 
    def __init__(self,url, num_pages): 
        self.url = url + str(num_pages)
        self.num_pages = num_pages
        self.page = BeautifulSoup(requests.get(self.url).text, "html.parser")
        self.movie_links = []
        for i in self.page.find_all(attrs={'class':'js-tile-link'}):
            try: 
                self.movie_links.append('https://www.rottentomatoes.com' + str(i.attrs['href']))
            except: 
                pass 
    
    def scrape_movie_data(self):
        for movie_link in self.movie_links: 
            movie = BeautifulSoup(requests.get(movie_link).text, "html.parser")
            print(movie)
            critics_score = movie.find_all(attrs={ 'class':'score-board-wrap'})
            print(critics_score)
            break 
            
            

    def scrape_movie_scores(self):
        print([item.text for item in self.page.find_all(attrs={'data-qa':'discovery-media-list-item-caption'})])
#             try: 
#                 self.movie_links.append('https://www.rottentomatoes.com' + str(i.attrs['href']))
#             except: 
#                 pass 
        
    
        

        

In [47]:
scraper_object = RottenTomatoesScraper('https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page=', 2)      

In [48]:
scraper_object.scrape_movie_scores()

['', '\n\n\n\r\n            The Ghosts of Monday\r\n        \n\r\n            Streaming Jan 23, 2023 \r\n          \n', '\n\n\n\r\n            M3GAN\r\n        \n\r\n            Streaming Jan 24, 2023 \r\n          \n', '\n\n\n\r\n            The Menu\r\n        \n\r\n            Streaming Jan 3, 2023 \r\n          \n', "\n\n\n\r\n            There's Something Wrong with the Children\r\n        \n\r\n            Streaming Jan 17, 2023 \r\n          \n", '\n\n\n\r\n            Vengeance\r\n        \n\r\n            Streaming Sep 16, 2022 \r\n          \n', '\n\n\n\r\n            Sick\r\n        \n\r\n            Streaming Jan 13, 2023 \r\n          \n', '\n\n\n\r\n            White Noise\r\n        \n\r\n            Streaming Dec 30, 2022 \r\n          \n', '\n\n\n\r\n            The Offering\r\n        \n\r\n            Streaming Jan 13, 2023 \r\n          \n', '\n\n\n\r\n            X\r\n        \n\r\n            Streaming Apr 14, 2022 \r\n          \n', '\n\n\n\r\n            Barbari